## Demonstration query against EMAP

Actually runs against `star_synth` courtesy of https://github.com/alan-turing-institute/sqlsynthgen

### Imports

In [1]:
import os
from pathlib import Path

import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from environs import Env

### Read in secrets etc. / Set up connection to EMAP

In [2]:
pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

env_parser = Env()
_project_root_dir = Path.cwd().resolve().parents[0].absolute()
env_file = _project_root_dir / ".env"
env_parser.read_env(env_file.as_posix(), override=True, recurse=False)

def emap_db() -> Engine:
    url = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
        os.getenv('EMAP_DB_USER'),
        os.getenv('EMAP_DB_PASSWORD'),
        os.getenv('EMAP_DB_HOST'),
        os.getenv('EMAP_DB_PORT'),
        os.getenv('EMAP_DB_NAME'),
    )
    engine = create_engine(
        url, pool_size=4, max_overflow=6, connect_args={"connect_timeout": 120}
    )
    return engine

emap_engine = emap_db()

### Demo SQL script (read core demographics)
```sql
SELECT 
 mrn_id
,date_of_birth ,date_of_death
,ethnicity
,firstname, lastname
,sex
,home_postcode
FROM star_synth.core_demographic LIMIT 5 ;
```

### Go fetch the data

In [3]:
query = """
SELECT 
 mrn_id, date_of_birth, date_of_death, ethnicity
,firstname, lastname, sex, home_postcode
FROM star_synth.core_demographic LIMIT 5;
"""
df = pd.read_sql_query(query, emap_engine)
print(df)

   mrn_id date_of_birth date_of_death               ethnicity  firstname lastname sex home_postcode
0    9243    1940-10-26    2015-05-28           White British     Parker   Castro   M      VV8W 0YK
1    9244    1939-08-25          None  Other White Background    Delinda  Pittman   M      RY4Y 6TL
2    9245    1953-07-05          None                    None    Voncile  Mathews   M      FM9G 3XM
3    9246    1989-05-14          None  Other White Background      Detra     Gill   F      JZ3R 1BH
4    9247    1950-02-08          None                    None  Nathaniel     Ford   F      BH8E 7LH
